In [1]:
from DataModule import MidiDataModule
from models.LSTMDiscriminator import LSTMDiscriminator
from models.LSTMGenerator import LSTMGenerator
from pytorch_lightning import LightningModule,Trainer
from torch.functional import F
from torch.optim import SGD
import torch,logging
import DownloadData
import TransformData

In [2]:
logging.basicConfig(filename="running.log", level=logging.INFO)

In [4]:
DownloadData.downloadDatasets(tmp_dir="../tmp/",unprocessed_data_dir='../data/unprocessed')
TransformData.transformData(unprocessed_data_dir='../data/unprocessed',processed_data_dir='../data/processed',csv_file='../data/index.csv')

 13%|█▎        | 557/4174 [00:57<06:15,  9.64it/s] 


KeyboardInterrupt: 

In [ ]:
data_module=MidiDataModule(csv_file="../data/index.csv",path_to_midi="../data/processed",batch_size=1,num_workers=0)

In [ ]:
data_module.setup('fit')
train_loader=data_module.train_dataloader()

In [ ]:
class GANLSTM(LightningModule):
    def __init__(self,**kwargs):
        super().__init__()
        self.generator=LSTMGenerator()
        self.discriminator=LSTMDiscriminator()
        self.save_hyperparameters()
    
    def forward(self,random_or_info):
        pitch,velocity,duration,step,info=self.generator(random_or_info)
        return pitch,velocity,duration,step,info
    
    def ad_loss(self,y_hat,y):
        return F.binary_cross_entropy(y_hat,y)

    def training_step(self, batch, batch_idx, optimizer_idx):
        # 'pitch_tensors', 'velocity_tensors', 'duration_tensors', 'step_tensors'
        length_of_batch=len(batch['pitch_tensors'])

        if optimizer_idx==0:
            
            
            # Generate random data
            random_data=None

            generated_data={'pitch_tensors':[],'velocity_tensors':[],'duration_tensors':[],'step_tensors':[]}

            # Train the generator
            for _ in range(length_of_batch):
                pitch,velocity,duration,step,random_data=self.generator(random_data)
                generated_data['pitch_tensors'].append(pitch)
                generated_data['velocity_tensors'].append(velocity)
                generated_data['duration_tensors'].append(duration)
                generated_data['step_tensors'].append(step)

            # Feed into the discriminator
            # pitches,velocities,durations,steps
            y_hat=self.discriminator(generated_data['pitch_tensors'],generated_data['velocity_tensors'],generated_data['duration_tensors'],generated_data['step_tensors'])
            y=torch.ones(y_hat.shape).type_as(y_hat)
            loss_g=self.ad_loss(y_hat,y)
            return {'loss':loss_g,'log':{'loss_g':loss_g}}
        
        elif optimizer_idx==1:
            
            # Real data
            

            y_hat=self.discriminator(batch['pitch_tensors'],batch['velocity_tensors'],batch['duration_tensors'],batch['step_tensors'])
           
            y=torch.ones(y_hat.shape).type_as(y_hat)
            real_loss=self.ad_loss(y_hat,y)
            
            
            # Fake data
            # Generate fake data
            random_data=None

            generated_data={'pitch_tensors':[],'velocity_tensors':[],'duration_tensors':[],'step_tensors':[]}

            # Train the generator
            for _ in range(length_of_batch):
                pitch,velocity,duration,step,random_data=self.generator(random_data)
                generated_data['pitch_tensors'].append(pitch)
                generated_data['velocity_tensors'].append(velocity)
                generated_data['duration_tensors'].append(duration)
                generated_data['step_tensors'].append(step)
            
            y_hat=self.discriminator(generated_data['pitch_tensors'],generated_data['velocity_tensors'],generated_data['duration_tensors'],generated_data['step_tensors'])
            y=torch.zeros(y_hat.shape).type_as(y_hat)
            
            fake_loss=self.ad_loss(y_hat,y)

            total_loss=real_loss+fake_loss
            total_loss=total_loss/2
            return {'loss':total_loss,'log':{'loss_d':total_loss,'loss_real':real_loss,'loss_fake':fake_loss}}
    
    def configure_optimizers(self):

        opt_g = SGD(self.generator.parameters(), lr=0.001, momentum=0.9)
        opt_d = SGD(self.discriminator.parameters(), lr=0.001, momentum=0.9)
        return [opt_g, opt_d], []


In [ ]:
gan_lstm=GANLSTM()
Trainer(accelerator="gpu").fit(gan_lstm,train_loader)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name          | Type              | Params
----------------------------------------------------
0 | generator     | LSTMGenerator     | 2.6 M 
1 | discriminator | LSTMDiscriminator | 2.9 M 
----------------------------------------------------
5.5 M     Trainable params
0         Non-trainable params
5.5 M     Total params
22.085    Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

Training Generator
info torch.Size([500])
Training Discriminator
info torch.Size([500])
info torch.Size([500])
Training Generator
info torch.Size([500])
Training Discriminator
info torch.Size([500])
info torch.Size([500])
Training Generator
info torch.Size([500])
Training Discriminator
info torch.Size([500])
